In [49]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Parser,
    Simulator,
    StartMeshcat,
    HalfSpace,
    CoulombFriction,
    PidController,
    LeafSystem, 
    BasicVector
)
import numpy as np

In [50]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://c140ec4a-6224-46c8-b55a-d8a7a25ad3a5.deepnoteproject.com/7003/


In [57]:
# Define a custom system for generating the desired trajectory
class LeftArmTrajectorySource(LeafSystem):
    def __init__(self):
        super().__init__()
        # Declare an output port for the desired state (position and velocity)
        self.DeclareVectorOutputPort("desired_state", BasicVector(4), self.CalcOutput)

    def CalcOutput(self, context, output):
        t = context.get_time()
        theta1 = 1.57 / 10 * np.sin(5*t)          # Target position
        theta1_dot = 1.57 / 10 * np.cos(5*t)      # Target velocity (derivative of sin)
        theta2 = 0          # Target position
        theta2_dot = 0      # Target velocity (derivative of sin)
        output.SetFromVector([theta1, theta2, theta1_dot, theta2_dot])

In [58]:
# Define a custom system for generating the desired trajectory
class RightArmTrajectorySource(LeafSystem):
    def __init__(self):
        super().__init__()
        # Declare an output port for the desired state (position and velocity)
        self.DeclareVectorOutputPort("desired_state", BasicVector(4), self.CalcOutput)

    def CalcOutput(self, context, output):
        t = context.get_time()
        theta1 = -1.57 / 10 * np.sin(5*t)          # Target position
        theta1_dot = -1.57 / 10 * np.cos(5*t)      # Target velocity (derivative of sin)
        theta2 = 0          # Target position
        theta2_dot = 0      # Target velocity (derivative of sin)
        output.SetFromVector([theta1, theta2, theta1_dot, theta2_dot])

In [43]:
class ExtractLeftArmStates(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Declare an input port for the full plant state (22 states)
        input_port = self.DeclareVectorInputPort("plant_state", BasicVector(22))
        
        # Declare an output port for the actuated states 
        self.DeclareVectorOutputPort(
            "lower_body_state", BasicVector(4),  # 4 states in total
            lambda context, output: output.SetFromVector(
                np.hstack([
                    self.get_input_port(0).Eval(context)[3:5],         # 4th to 5th states
                    self.get_input_port(0).Eval(context)[14:16],       # 15th to 16th states
                ])
            )
        )

In [44]:
class ExtractRightArmStates(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Declare an input port for the full plant state (22 states)
        input_port = self.DeclareVectorInputPort("plant_state", BasicVector(22))
        
        # Declare an output port for the actuated states 
        self.DeclareVectorOutputPort(
            "lower_body_state", BasicVector(4),  # 4 states in total
            lambda context, output: output.SetFromVector(
                np.hstack([
                    self.get_input_port(0).Eval(context)[5:7],         # 6th to 7th states
                    self.get_input_port(0).Eval(context)[16:18],       # 17th to 18th states
                ])
            )
        )

In [45]:
class ExtendInputs(LeafSystem):
    def __init__(self):
        super().__init__()
        
        # Declare a single input port that takes a 4-dimensional vector
        self.DeclareVectorInputPort("left_arm_input_vector", BasicVector(2))
        self.DeclareVectorInputPort("right_arm_input_vector", BasicVector(2))
        
        # Declare an output port with 8 elements
        self.DeclareVectorOutputPort(
            "combined_output", BasicVector(8),
            # lambda context, output: output.SetFromVector(
            #     [0, 0, 0, 0, self.get_input_port(0).Eval(context)[0], 0, self.get_input_port(0).Eval(context)[1], 0]
            # )
            lambda context, output: output.SetFromVector(
                [0, 0, 0, 0, self.get_input_port(0).Eval(context)[0], 
                self.get_input_port(1).Eval(context)[0], 
                self.get_input_port(0).Eval(context)[1], 
                self.get_input_port(1).Eval(context)[1]]
            )
        )

In [59]:
def animation_demo():
    builder = DiagramBuilder()

    # Adds both MultibodyPlant and the SceneGraph, and wires them together.
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    X_WG = HalfSpace.MakePose(np.array([0,0, 1]), np.zeros(3,))
    plant.RegisterCollisionGeometry(
        plant.world_body(), 
        X_WG, HalfSpace(), 
        "collision", 
        CoulombFriction(1.0, 1.0)
    )
    # Note that we parse into both the plant and the scene_graph here.
    Parser(plant, scene_graph).AddModels("humanoid_walker.urdf")
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base"))
    plant.Finalize()

    # Secondary MultibodyPlant for control
    control_plant = MultibodyPlant(time_step=1e-4)
    control_model = Parser(control_plant).AddModels("double_pendulum.urdf")[0]
    control_plant.Finalize()

    # Add MeshcatVisualizer for visualization
    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

    # Define PID gains and add the inverse dynamics controller
    kp = [100] * control_plant.num_positions()
    ki = [1] * control_plant.num_positions()
    kd = [20] * control_plant.num_positions()
    left_arm_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))
    left_arm_controller.set_name("left_arm_controller")
    right_arm_controller = builder.AddSystem(InverseDynamicsController(control_plant, kp, ki, kd, False))
    right_arm_controller.set_name("right_arm_controller")

    left_arm_system = ExtractLeftArmStates()
    right_arm_system = ExtractRightArmStates()
    builder.AddSystem(left_arm_system)
    builder.AddSystem(right_arm_system)

    # wire plant state to actuated system
    builder.Connect(plant.get_state_output_port(), 
                    left_arm_system.get_input_port(0))
    builder.Connect(plant.get_state_output_port(), 
                    right_arm_system.get_input_port(0))

    builder.Connect(
        left_arm_system.get_output_port(0),  
        left_arm_controller.get_input_port_estimated_state(),
    )
    builder.Connect(
        right_arm_system.get_output_port(0),  
        right_arm_controller.get_input_port_estimated_state(),
    )

    full_body_input = ExtendInputs()
    builder.AddSystem(full_body_input)

    builder.Connect(left_arm_controller.get_output_port(), 
                    full_body_input.get_input_port(0))
    builder.Connect(right_arm_controller.get_output_port(), 
                    full_body_input.get_input_port(1))
    builder.Connect(full_body_input.get_output_port(0), 
                    plant.get_actuation_input_port())

    # Create and add the trajectory source to generate the desired trajectory
    left_arm_trajectory_source = builder.AddSystem(LeftArmTrajectorySource())
    right_arm_trajectory_source = builder.AddSystem(RightArmTrajectorySource())
    builder.Connect(
        left_arm_trajectory_source.get_output_port(0), 
        left_arm_controller.get_input_port_desired_state()
    )
    builder.Connect(
        right_arm_trajectory_source.get_output_port(0), 
        right_arm_controller.get_input_port_desired_state()
    )

    # Build the diagram
    diagram = builder.Build()
    diagram.set_name("humanoid_arm_controller")

    # Set initial conditions
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyMutableContextFromRoot(context)
    q0 = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    plant.SetPositions(plant_context, q0)

    # Run the simulation
    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)

    # Start recording the animation
    meshcat.StartRecording()
    simulator.AdvanceTo(1.5)
    meshcat.StopRecording()
    meshcat.PublishRecording()

animation_demo()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c140ec4a-6224-46c8-b55a-d8a7a25ad3a5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>